## Libraries and Modules

In [ ]:
import glob
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import cv2
from tqdm.auto import tqdm
import os
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing import image
from keras.models import Model
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import smart_resize
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image, display
from tensorflow import keras
!pip install scipy
import scipy
import pylab as pl
from PIL import Image
import json
import seaborn as sns
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

images_path = "/content/drive/MyDrive/Image-Clustering-v3/Screenshots/"
img_names = list(glob.glob("/content/drive/MyDrive/Image-Clustering-v3/Screenshots/"+'*.jpg'))

## Functions

### Get Embeddings

In [ ]:
def z_norm(df,epsilon=10e-10):
    return (df-df.mean())/(df.std()+epsilon)

def min_max_norm(df,epsilon=10e-10):
    return (df-df.min())/(df.max()-df.min()+epsilon)

def exp_norm(df,epsilon=10e-10):
    em = df.to_numpy()
    mean = np.mean(em,axis=0)
    lam = 1/(mean+epsilon)
    exfit = lam*np.exp(-lam*em)
    exfit = (exfit - np.min(exfit,axis=0)) / (np.max(exfit,axis=0)-np.min(exfit,axis=0)+epsilon)
    return pd.DataFrame(exfit,index=df.index)

In [ ]:
def extract_features(file, model,input_shape):
    img = image.load_img(file)
    img = image.img_to_array(img)
    img = smart_resize(img, (input_shape,input_shape), interpolation='bilinear')
    img = np.expand_dims(img, axis=0)
    features = model.predict(img, use_multiprocessing=True)
    return features.ravel()

def frame_embeddings(data,normalize=None):
    embedding_df = pd.DataFrame.from_dict(data)
    embedding_df = embedding_df.T
    print(embedding_df.shape)
    if normalize is not None : embedding_df = normalize(embedding_df)
    embedding_df['image'] = embedding_df.index
    embedding_df = embedding_df.reset_index()
    embedding_df = embedding_df.drop(['index'], axis=1)
    return embedding_df

def get_embeddings(model,input_shape,normalize=None):
    data = {}
    for img in img_names:
      feat = extract_features(img,model,input_shape)
      data[img] = feat
    embedding_df = frame_embeddings(data,normalize)
    return embedding_df

### Dimensionality Reduction

In [ ]:
def dimension_reduction(embedding_df,components):
  pca = PCA(n_components=components)
  pca.fit(embedding_df.drop(columns='image'))
  p = pca.transform(embedding_df.drop(columns='image'))
  print('###############################################')
  print('number of pca components',len(pca.components_))
  print('###############################################')
  
  return p

### Clustering

In [ ]:
def image_clustering(embedding_df,principal_cp,algorithm,return_stats=False):
    cluster       = algorithm.fit(principal_cp)
    labels        = cluster.labels_
    classified    = len([x for x in labels if x>=0])
    num_cls       = len(np.unique(labels))
    unique_counts = np.unique(labels, return_counts=True)
    avg_cls       = np.mean(np.where(unique_counts[0] > -1,unique_counts[1],0))
    std_cls       = np.std( unique_counts[1] )
    print('___________________________________________________________________')
    print('number of clusters    : ',num_cls)
    print('classified points     : ',classified)
    print('average cluster pop   : ',avg_cls)
    print('standard deviation    : ',std_cls)
    curr_df = pd.DataFrame()
    curr_df['cluster'] = labels
    df = pd.concat([embedding_df["image"],curr_df["cluster"]], axis=1)
    if return_stats==True:
      dic = {}
      dic['clusters'] = num_cls
      dic['coverage'] = classified
      dic['avg']      = avg_cls
      dic['std']      = std_cls
      return df,dic

    return df,classified, avg_cls

### Experiment

In [ ]:
def experiment(model,input_shape,normalize,components,algorithm):
  embedding_df = get_embeddings(model,input_shape,normalize)
  principal_cp = dimension_reduction(embedding_df,components)
  df,classes,avg_pop = image_clustering(embedding_df,principal_cp,algorithm)
  return df,classes,avg_pop

def experiment_loop(model,input_shape,normalize,components,min_samples,xi):
  embedding_df = get_embeddings(model,input_shape,normalize)
  class_grid = []
  avpop_grid = []
  for i in range(components[0],components[1],components[2]):
    crow = []
    prow = []
    principal_cp = dimension_reduction(embedding_df,i)
    for j in range(min_samples[0],min_samples[1],min_samples[2]):
      ccol = []
      pcol = []
      for k in np.arange(xi[0],xi[1],xi[2]):
        algorithm   = OPTICS(min_samples=j, xi=k)
        df,classes,avg_pop = image_clustering(embedding_df,principal_cp,algorithm)
        print(i,j,k)
        ccol.append(classes)
        pcol.append(avg_pop)
      crow.append(ccol)
      prow.append(pcol)
    class_grid.append(crow)
    avpop_grid.append(prow)
  return class_grid, avpop_grid


def stats_experiment_loop(model,input_shape,normalize,components,min_samples,xi):
  embedding_df = get_embeddings(model,input_shape,normalize)
  grid = []
  for i in range(components[0],components[1],components[2]):
    row = []
    principal_cp = dimension_reduction(embedding_df,i)
    for j in range(min_samples[0],min_samples[1],min_samples[2]):
      col = []
      for k in np.arange(xi[0],xi[1],xi[2]):
        algorithm   = OPTICS(min_samples=j, xi=k)
        df,stats = image_clustering(embedding_df,principal_cp,algorithm,return_stats=True)
        print(i,j,k)
        stats['stats']=str(i)+'_'+str(j)+'_'+str(k)
        col.append(stats)
      row.append(col)
    grid.append(row)
  return grid

### Utilities

In [ ]:
def save_grid(mat,name):
  with open(name, "w") as fp:
    json.dump(mat, fp)
  return

def read_grid(name):
  with open(name, "r") as fp:
    grid = json.load(fp)
  return grid
  
def show_contour(grid,dim1,dim2):
  if dim1+dim2   == 1: grid = np.mean(grid,axis=2)
  elif dim1+dim2 == 2: grid = np.mean(grid,axis=1)
  elif dim1+dim2 == 3: grid = np.mean(grid,axis=0)

  print(grid.shape)

  X, Y = np.meshgrid(np.linspace(0,1,grid.shape[1]), np.linspace(0,1,grid.shape[0]))
  plt.contourf(X,Y,grid,cmap='gist_rainbow_r')
  plt.show()
  return

def open_image(image):
  image = Image.open(image)
  plt.imshow(image)
  plt.show()
  return

def clust_in_df(df,clust_id):
  return df.loc[df['cluster'] == clust_id]["image"]


def search_stat_grid(grid,ncls,cov,avg,std,return_list=False):
  lis = []
  counter = 1
  for i in range(len(grid)):
    for j in range(len(grid[i])):
      for k in range(len(grid[i][j])):
        #print(grid[i][j][k])
        if ncls[0] > grid[i][j][k]['clusters'] or grid[i][j][k]['clusters'] > ncls[1]:
          continue
        if cov[0] > grid[i][j][k]['coverage'] or grid[i][j][k]['coverage'] > cov[1]:
          continue
        if avg[0] > grid[i][j][k]['avg'] or grid[i][j][k]['avg'] > avg[1]:
          continue
        if std[0] > grid[i][j][k]['std'] or grid[i][j][k]['std'] > std[1]:
          continue
        if return_list == True:
          lis.append(grid[i][j][k])
          continue
        print(counter,grid[i][j][k])
        counter += 1
  if return_list == True: return lis
  return


##Run

In [ ]:
#                                                                           Parameters
input_shape = 224
model       = keras.applications.ResNet50V2(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
algorithm   = OPTICS(min_samples=3, xi=0.05)
components  = 15
epsilon     = 10e-10
norm_func   = z_norm

### Grid Search

In [ ]:
class_grid, avpop_grid = experiment_loop(model,input_shape,None,[5,25,5],[2,7,1],[0.05,0.1,0.01])

In [ ]:
print(np.array(class_grid).shape)
print('xi independent')
show_contour(avpop_grid,0,1)
print('pca independent')
show_contour(avpop_grid,1,2)
print('min independent')
show_contour(avpop_grid,2,0)

print('xi independent')
show_contour(class_grid,0,1)
print('pca independent')
show_contour(class_grid,1,2)
print('min independent')
show_contour(class_grid,2,0)

In [ ]:
cl = np.array(class_grid).reshape(1,-1).ravel().tolist()
cl.sort(reverse=True)
for each in cl:
  print()
  print(each,'->',np.where(np.array(class_grid)==each))

### Single Experiment

In [ ]:
df = experiment(model,input_shape,z_norm,components,algorithm)

1/1 [==============================] - 0s 18ms/step
(838, 512)
###############################################
number of pca components 20
###############################################
___________________________________________________________________
number of clusters    :  18
classified points     :  64
average cluster pop   :  46.55555555555556
standard deviation    :  176.43482341527775


In [ ]:
for clust_no in range(0,10):
  print("##################",clust_no,"##################")
  clust_list = list(clust_in_df(df[0],clust_no))
  for im in clust_list: open_image(im)

##Correction Experiment

### Multi Model Correction

In [ ]:
#                                                                           Parameters
input_shape = 224
model       = keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
algorithm   = OPTICS(min_samples=3, xi=0.03)
components  = 20
epsilon     = 10e-10

In [ ]:
def rem_image_col(lis):
  return [ x.drop(columns='image') for x in lis ]
mod1  = keras.applications.VGG16(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
mod2  = keras.applications.VGG19(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
mod7  = keras.applications.MobileNet(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
mod8  = keras.applications.MobileNetV2(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
mod9  = keras.applications.NASNetMobile(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')


edf0           = get_embeddings(model,input_shape,None).reset_index()
model_list     = [mod1,mod2,mod7,mod8,mod9]
embedding_list = [get_embeddings(m,input_shape,None).reset_index().drop(columns='image') for m in model_list]
embedding_list.append(edf0)

In [ ]:
embedding_df = pd.concat(embedding_list, axis=1)
embedding_df = embedding_df.drop(columns='index')
embedding_df

In [ ]:
em = embedding_df
em = em.drop(columns='image')
em = em.to_numpy()
mean = np.sum(em,axis=0)/em.shape[0]
lam  = 1/(mean+epsilon)
exfit = lam*np.exp(-lam*em)
exfit = (exfit - np.min(exfit,axis=0)) / (np.max(exfit,axis=0)-np.min(exfit,axis=0)+epsilon)

In [ ]:
ex = pd.DataFrame(exfit)
ex['image'] = embedding_df['image']
principal_cp = dimension_reduction(embedding_df,components)
df,classes,avg_pop = image_clustering(embedding_df,principal_cp,algorithm)

In [ ]:
for clust_no in range(0,30):
  print("##################",clust_no,"##################")
  clust_list = list(clust_in_df(df,clust_no))
  for im in clust_list: open_image(im)

In [ ]:
print(len(df))
'''
for i in range(len(df)):
  print(df.loc[i])
'''

for each in list(df.loc[0]): print(each)

### Corrected Grid Experiment 

In [ ]:
5 5 0.03
5 6 0.01

In [ ]:
#                                                                           Parameters
input_shape = 224
model       = keras.applications.VGG16(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
epsilon     = 10e-10

In [ ]:
grid = stats_experiment_loop(model,input_shape,exp_norm,[1,10,1],[2,9,1],[0.01,0.1,0.01])
save_grid(grid,'corrected_stat_grid')

In [ ]:
grid = read_grid('corrected_stat_grid')
search_stat_grid(grid,[0,40],[0,1000],[0,100],[1,100],return_list=False)

In [ ]:
res = search_stat_grid(grid,[0,30],[200,1000],[0,800],[1,1000],return_list=True)
res = sorted(res, key=lambda x: x['clusters'], reverse=True)
for each in res: print(each)

### Single Correction Experiment

In [ ]:
#                                                                           Parameters
input_shape = 224
model       = keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
algorithm   = OPTICS(min_samples=6, xi=0.01)
components  = 5
epsilon     = 10e-10

In [ ]:
embedding_df           = get_embeddings(model,input_shape,exp_norm)
embedding_df

In [ ]:
principal_cp = dimension_reduction(embedding_df,components)
df,classes,avg_pop = image_clustering(embedding_df,principal_cp,algorithm)

In [ ]:
lis = []
for clust_no in range(0,30):
  print("##################",clust_no,"##################")
  clust_list = list(clust_in_df(df,clust_no))
  lis.append(len(clust_list))
  for im in clust_list: open_image(im)
print(lis)

In [ ]:
lis = []
for clust_no in range(0,100):
  clust_list = list(clust_in_df(df,clust_no))
  if len(clust_list)>0: lis.append(len(clust_list))
print(lis)

### Correction Evidence

In [ ]:
#                                                                           Parameters
input_shape = 224
model       = keras.applications.VGG16(include_top=False,weights='imagenet',input_shape=(input_shape,input_shape,3),pooling='avg')
algorithm   = OPTICS(min_samples=3, xi=0.05)
components  = 20
epsilon     = 10e-10

In [ ]:
embedding_df           = get_embeddings(model,input_shape,False).reset_index()

In [ ]:
for i in range(10):
  lam = 1/(embedding_df[i].mean()+10e-10)
  x = np.linspace(0,embedding_df[i].max(),len(embedding_df[i]))
  lis = list(embedding_df[i])
  lis.sort()
  plt.plot(x,lam*np.exp(-lam*x))
  plt.hist(lis,bins=100,density=True)
  plt.show()

In [ ]:
embedding_df.mean()

In [ ]:
embedding_df

In [ ]:
if normalize is not None : embedding_df = normalize(embedding_df)
    embedding_df['image'] = embedding_df.index
    embedding_df = embedding_df.reset_index()
    embedding_df = embedding_df.drop(['index'], axis=1)